In [1]:
# %pip install rasterio

In [46]:
import os
import sys
import random
import numpy as np
from tqdm import tqdm
from os.path import dirname as up

import torch
from torch import nn
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from sklearn.metrics import (f1_score, precision_score, recall_score, accuracy_score, 
                             jaccard_score, hamming_loss, label_ranking_loss, coverage_error)
import sklearn.metrics as metr
import pandas as pd
from tqdm import tqdm
from osgeo import gdal
from os.path import dirname as up
from torch.utils.data import Dataset
import torchvision.transforms.functional as F

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss(ignore_index=-1)(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

# Model Making

In [43]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)


In [44]:
class ResidualBlock(nn.Module):
    def __init__(self, inputChannel, outputChannel, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(inputChannel, outputChannel, stride)
        self.bn1 = nn.BatchNorm2d(outputChannel)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(outputChannel, outputChannel)
        self.bn2 = nn.BatchNorm2d(outputChannel)
        self.downsample = downsample
        self.ca = ChannelAttention(outputChannel)
        self.sa = SpatialAttention()
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        caOutput = self.ca(out)
        out = caOutput * out
        saOutput = self.sa(out)
        out = saOutput * out
        return out, saOutput


class DownSampleWithAttention(nn.Module):
    def __init__(self, inputChannel, outputChannel):
        super().__init__()
        self.convolution = nn.Sequential(
            nn.Conv2d(inputChannel, outputChannel, kernel_size=3, padding=1),
            nn.BatchNorm2d(outputChannel),
            nn.LeakyReLU(0.2),
            nn.Conv2d(outputChannel, outputChannel, kernel_size=3, padding=1),
            nn.BatchNorm2d(outputChannel),
            nn.LeakyReLU(0.2),
            nn.AvgPool2d(2)
        )
        self.ca = ChannelAttention(outputChannel)
        self.sa = SpatialAttention()
    
    def forward(self,x):
        x = self.convolution(x)
        caOutput = self.ca(x)
        x = caOutput * x
        saOutput = self.sa(x)
        x = saOutput * x
        return x, saOutput


In [45]:
class UpSampleWithAttention(nn.Module):
    def __init__(self, inputChannel, outputChannel):
        super().__init__()
        self.convolution = nn.Sequential(
            nn.Conv2d(inputChannel, outputChannel, kernel_size=3, padding=1),
            nn.BatchNorm2d(outputChannel),
            nn.LeakyReLU(0.2),
            nn.Conv2d(outputChannel, outputChannel, kernel_size=3, padding=1),
            nn.BatchNorm2d(outputChannel),
            nn.LeakyReLU(0.2)
        )
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.ca = ChannelAttention(outputChannel)
        self.sa = SpatialAttention()
    
    def forward(self, x):
        x = self.upsample(x)
        x = self.convolution(x)
        caOutput = self.ca(x)
        x = caOutput * x
        saOutput = self.sa(x)
        x = saOutput * x
        return x, saOutput
    
class ResidualAttentionUNet(nn.Module):
  def __init__(self, inputChannel, outputChannel):
    super().__init__()
    self.downsample1 = DownSampleWithAttention(inputChannel, 32)
    self.downsample2 = DownSampleWithAttention(32, 64)
    self.downsample3 = DownSampleWithAttention(64, 128)
    self.downsample4 = DownSampleWithAttention(128, 256)
    self.downsample5 = DownSampleWithAttention(256, 512)

    self.residualBlock1 = ResidualBlock(512, 512)
    self.residualBlock2 = ResidualBlock(512, 512)
    self.residualBlock3 = ResidualBlock(512, 512)

    self.upsample1 = UpSampleWithAttention(512, 256)
    self.upsample2 = UpSampleWithAttention(512, 128)
    self.upsample3 = UpSampleWithAttention(256, 64)
    self.upsample4 = UpSampleWithAttention(128, 32)
    self.upsample5 = UpSampleWithAttention(64, 32)
    self.classification = nn.Sequential(
            nn.Conv2d(32, outputChannel, kernel_size=1),
        )

  def forward(self, x):
    scale128, sa128down = self.downsample1(x)
    scale64, sa64down = self.downsample2(scale128)
    scale32, sa32down = self.downsample3(scale64)
    scale16, sa64down = self.downsample4(scale32)
    scale8, sa8down = self.downsample5(scale16)
    scale8, sa8down = self.residualBlock1(scale8)
    scale8, sa8down = self.residualBlock2(scale8)
    scale8, sa8down = self.residualBlock3(scale8)
    upscale16, sa16up = self.upsample1(scale8)
    upscale16 = torch.cat([upscale16, scale16], dim=1)
    upscale32, sa32up = self.upsample2(upscale16)
    upscale32 = torch.cat([upscale32, scale32], dim=1)
    upscale64, sa64up = self.upsample3(upscale32)
    upscale64 = torch.cat([upscale64, scale64], dim=1)
    upscale128, sa128up = self.upsample4(upscale64)
    upscale128 = torch.cat([upscale128, scale128], dim=1)
    upscale256, sa256up = self.upsample5(upscale128)
    finaloutput = self.classification(upscale256)
    return finaloutput

# Metrics

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
def Evaluation(y_predicted, y_true):

    micro_prec = precision_score(y_true, y_predicted, average='micro')
    macro_prec = precision_score(y_true, y_predicted, average='macro')
    weight_prec = precision_score(y_true, y_predicted, average='weighted')
    
    micro_rec = recall_score(y_true, y_predicted, average='micro')
    macro_rec = recall_score(y_true, y_predicted, average='macro')
    weight_rec = recall_score(y_true, y_predicted, average='weighted')
        
    macro_f1 = f1_score(y_true, y_predicted, average="macro")
    micro_f1 = f1_score(y_true, y_predicted, average="micro")
    weight_f1 = f1_score(y_true, y_predicted, average="weighted")
        
    subset_acc = accuracy_score(y_true, y_predicted)
    
    iou_acc = jaccard_score(y_true, y_predicted, average='macro')

    info = {
            "macroPrec" : macro_prec,
            "microPrec" : micro_prec,
            "weightPrec" : weight_prec,
            "macroRec" : macro_rec,
            "microRec" : micro_rec,
            "weightRec" : weight_rec,
            "macroF1" : macro_f1,
            "microF1" : micro_f1,
            "weightF1" : weight_f1,
            "subsetAcc" : subset_acc,
            "IoU": iou_acc
            }
    
    return info

In [27]:
def confusion_matrix(y_gt, y_pred, labels):
    cm      = metr.confusion_matrix  (y_gt, y_pred)
    f1_macro= metr.f1_score          (y_gt, y_pred, average='macro')
    mPA      = metr.recall_score      (y_gt, y_pred, average='macro')
    OA      = metr.accuracy_score    (y_gt, y_pred)
    UA      = metr.precision_score   (y_gt, y_pred, average=None)
    PA      = metr.recall_score      (y_gt, y_pred, average=None)
    f1      = metr.f1_score          (y_gt, y_pred, average=None)
    IoC     = metr.jaccard_score     (y_gt, y_pred, average=None)
    mIoC     = metr.jaccard_score    (y_gt, y_pred, average='macro')
    sz1, sz2 = cm.shape
    cm_with_stats             = np.zeros((sz1+4,sz2+2))
    cm_with_stats[0:-4, 0:-2] = cm
    cm_with_stats[-3  , 0:-2] = np.round(IoC,2)
    cm_with_stats[-2  , 0:-2] = np.round(UA,2)
    cm_with_stats[-1  , 0:-2] = np.round(f1,2)
    cm_with_stats[0:-4,   -1] = np.round(PA,2)
    
    cm_with_stats[-4  , 0:-2] = np.sum(cm, axis=0) 
    cm_with_stats[0:-4,   -2] = np.sum(cm, axis=1)
    cm_list = cm_with_stats.tolist()
    first_row = []
    first_row.extend (labels)
    first_row.append ('Sum')
    first_row.append ('Recall')
    first_col = []
    first_col.extend(labels)
    first_col.append ('Sum')
    first_col.append ('IoU')
    first_col.append ('Precision')
    first_col.append ('F1-score')
    idx = 0
    for sublist in cm_list:
        if   idx == sz1:
            sublist[-2]  = 'mPA:'
            sublist[-1]  = round(mPA,2)
            cm_list[idx] = sublist
        elif   idx == sz1+1:
            sublist[-2]  = 'mIoU:'
            sublist[-1]  = round(mIoC,2)
            cm_list[idx] = sublist
            
        elif idx == sz1+2:
            sublist[-2]  = 'OA:'
            sublist[-1]  = round(OA,2)
            cm_list[idx] = sublist
            
        elif idx == sz1+3:
            cm_list[idx] = sublist
            sublist[-2]  = 'F1-macro:'
            sublist[-1]  = round(f1_macro,2)    
        idx +=1
    df = pd.DataFrame(np.array(cm_list))
    df.columns = first_row
    df.index = first_col
    
    return df

# Dataloader

In [8]:
class_distr = torch.Tensor([0.00452, 0.00203, 0.00254, 0.00168, 0.00766, 0.15206, 0.20232,
0.35941, 0.00109, 0.20218, 0.03226, 0.00693, 0.01322, 0.01158, 0.00052])

bands_mean = np.array([0.05197577, 0.04783991, 0.04056812, 0.03163572, 0.02972606, 0.03457443,
0.03875053, 0.03436435, 0.0392113,  0.02358126, 0.01588816]).astype('float32')

bands_std = np.array([0.04725893, 0.04743808, 0.04699043, 0.04967381, 0.04946782, 0.06458357,
0.07594915, 0.07120246, 0.08251058, 0.05111466, 0.03524419]).astype('float32')


In [9]:
dataset_path = 'data'

class GenDEBRIS(Dataset): 
    def __init__(self, mode = 'train', transform=None, standardization=None, path = dataset_path, agg_to_water= True):
        
        if mode=='train':
            self.ROIs = np.genfromtxt(os.path.join(path, 'splits', 'train_X.txt'),dtype='str')
                
        elif mode=='test':
            self.ROIs = np.genfromtxt(os.path.join(path, 'splits', 'test_X.txt'),dtype='str')
                
        elif mode=='val':
            self.ROIs = np.genfromtxt(os.path.join(path, 'splits', 'val_X.txt'),dtype='str')
            
        else:
            raise
            
        self.X = []           
        self.y = []           
            
        for roi in tqdm(self.ROIs, desc = 'Load '+mode+' set to memory'):
            roi_folder = '_'.join(['S2'] + roi.split('_')[:-1])               
            roi_name = '_'.join(['S2'] + roi.split('_'))                      
            roi_file = os.path.join(path, 'patches', roi_folder,roi_name + '.tif')       
            roi_file_cl = os.path.join(path, 'patches', roi_folder,roi_name + '_cl.tif') 
            
            
            ds = gdal.Open(roi_file_cl)
            temp = np.copy(ds.ReadAsArray().astype(np.int64))
            
            
            if agg_to_water:
                temp[temp==15]=7          
                temp[temp==14]=7          
                temp[temp==13]=7          
                temp[temp==12]=7          
            
            
            temp = np.copy(temp - 1)
            ds=None                   
            
            self.y.append(temp)
            
            
            ds = gdal.Open(roi_file)
            temp = np.copy(ds.ReadAsArray())
            ds=None
            self.X.append(temp)          

        self.impute_nan = np.tile(bands_mean, (temp.shape[1],temp.shape[2],1))
        self.mode = mode
        self.transform = transform
        self.standardization = standardization
        self.length = len(self.y)
        self.path = path
        self.agg_to_water = agg_to_water
        
    def __len__(self):

        return self.length
    
    def getnames(self):
        return self.ROIs
    
    def __getitem__(self, index):
        
        img = self.X[index]
        target = self.y[index]

        img = np.moveaxis(img, [0, 1, 2], [2, 0, 1]).astype('float32')       
        
        nan_mask = np.isnan(img)
        img[nan_mask] = self.impute_nan[nan_mask]
        
        if self.transform is not None:
            target = target[:,:,np.newaxis]
            stack = np.concatenate([img, target], axis=-1).astype('float32') 
        
            stack = self.transform(stack)

            img = stack[:-1,:,:]
            target = stack[-1,:,:].long()                                    
        
        if self.standardization is not None:
            img = self.standardization(img)
            
        return img, target

In [10]:
class RandomRotationTransform:
    def __init__(self, angles):
        self.angles = angles

    def __call__(self, x):
        angle = random.choice(self.angles)
        return F.rotate(x, angle)

In [11]:
def gen_weights(class_distribution, c = 1.02):
    return 1/torch.log(c + class_distribution)

# Training Code

In [47]:
labels = ['Marine Debris','Dense Sargassum','Sparse Sargassum',
          'Natural Organic Material','Ship','Clouds','Marine Water','Sediment-Laden Water',
          'Foam','Turbid Water','Shallow Water','Waves','Cloud Shadows','Wakes',
          'Mixed Water']

In [12]:
batchSizeTrain = 8  
batchSizeTest = 4  
trainOnGPU = True  
totalEpochs = 35  
logPath = "final_logs"  
initialLR = 1e-3  
decayLR = 0  
schedulerLR = "ms"  
trainOnMac = False  
useFocalLoss = False  
modelName = "resattunet"  
bestValidationAccuracy = 0.0

logPath = "./"+logPath
os.makedirs(logPath)
writer = SummaryWriter(logPath)

In [21]:
totalEpochs = 35

In [13]:
def seedAll(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def seedWorker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [14]:
seedAll(0)
g = torch.Generator()
g.manual_seed(0)
agg_to_water = True

transformTrain = transforms.Compose([transforms.ToTensor(),
                                    RandomRotationTransform([-90, 0, 90, 180]),
                                    transforms.RandomHorizontalFlip()])
    
transformTest = transforms.Compose([transforms.ToTensor()])
    
standardization = transforms.Normalize(bands_mean, bands_std)

datasetTrain = GenDEBRIS('train', transform=transformTrain, standardization = standardization, agg_to_water = agg_to_water)
datasetTest = GenDEBRIS('val', transform=transformTest, standardization = standardization, agg_to_water = agg_to_water)
        
trainLoader = DataLoader(datasetTrain, 
                        batch_size = batchSizeTrain, 
                        shuffle = True,
                        worker_init_fn=seedWorker,
                        generator=g)
        
testLoader = DataLoader(datasetTest, 
                        batch_size = batchSizeTest, 
                        shuffle = False,
                        worker_init_fn=seedWorker,
                        generator=g)

Load train set to memory:   0%|          | 0/694 [00:00<?, ?it/s]c:\Users\UZAIR KABEER\anaconda3\lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
Load val set to memory: 100%|██████████| 328/328 [00:04<00:00, 70.19it/s] 


In [15]:
device = torch.device("mps" if trainOnMac else ("cuda" if trainOnGPU else "cpu"))
model = {"resattunet": ResidualAttentionUNet(11, 11)}.get(modelName)

In [16]:
device

device(type='cuda')

In [17]:
if model is None:
    print("Enter correct choice of architecture")
    sys.exit()
model.to(device)

ResidualAttentionUNet(
  (downsample1): DownSampleWithAttention(
    (convolution): Sequential(
      (0): Conv2d(11, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.2)
      (6): AvgPool2d(kernel_size=2, stride=2, padding=0)
    )
    (ca): ChannelAttention(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (max_pool): AdaptiveMaxPool2d(output_size=1)
      (fc): Sequential(
        (0): Conv2d(32, 2, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): ReLU()
        (2): Conv2d(2, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (sigmoid): Sigmoid()
    )
    (sa): SpatialAttention(
      (conv1): Conv2d(2, 1, kern

In [18]:
if agg_to_water:
    agg_distr = sum(class_distr[-4:])
    class_distr[6] += agg_distr
    class_distr = class_distr[:-4]

if useFocalLoss:
    criterion = FocalLoss()
else:
    weight = gen_weights(class_distr, c = 1.03)
    criterion = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction= 'mean', weight=weight.to(device))

optimizer = torch.optim.Adam(model.parameters(), lr=initialLR, weight_decay=decayLR)

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=initialLR, weight_decay=decayLR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True) if schedulerLR == "rop" else torch.optim.lr_scheduler.MultiStepLR(optimizer, [40, 80, 120, 160], gamma=0.5, verbose=True)

c:\Users\UZAIR KABEER\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [22]:
bestMacroF1 = 0.0
bestMicroF1 = 0.0
bestWeightF1 = 0.0

i = 0
for epoch in range(1, totalEpochs+1):
    trainingBatches = 0
    model.train()
    print("Training for epoch:",epoch)
    for (image, target) in tqdm(trainLoader):
        image = image.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        logits = model(image)
        loss = criterion(logits, target)
        loss.backward()
        trainingBatches+=target.shape[0]
        writer.add_scalar('Training Loss', loss, i)
        i = i + 1
        optimizer.step()
    print("Completed epoch:",epoch)
    print("Validating model")
    model.eval()
    testBatches = 0
    yTrue = []
    yPredicted = []
    testLossF = []
    with torch.no_grad():
        for (image, target) in testLoader:
            image = image.to(device)
            target = target.to(device)
            logits = model(image)
            loss = criterion(logits, target)
            logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
            logits = logits.reshape((-1,11))
            target = target.reshape(-1)
            mask = target != -1
            logits = logits[mask]
            target = target[mask]
            probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
            target = target.cpu().numpy()
            testBatches += target.shape[0]
            testLossF.append((loss.data*target.shape[0]).tolist())
            yPredicted += probs.argmax(1).tolist()
            yTrue += target.tolist()
        writer.add_scalar('Testing Loss', sum(testLossF)/testBatches, epoch)
        yPredicted = np.asarray(yPredicted)
        yTrue = np.asarray(yTrue)
        acc = Evaluation(yPredicted, yTrue)
        modelname = logPath +"/intermediateModel.pth"
        torch.save(model.state_dict(), modelname)
        print("Test Macro Precision",acc["macroPrec"])
        writer.add_scalar('Test Macro Precision', acc["macroPrec"], epoch)
        writer.add_scalar('Test Micro Precision', acc["microPrec"], epoch)
        writer.add_scalar('Test Weight Precision', acc["weightPrec"], epoch)

        print("Test Macro Recall",acc["macroPrec"])
        writer.add_scalar('Test Macro Recall', acc["macroRec"], epoch)
        writer.add_scalar('Test Micro Recall', acc["microRec"], epoch)
        writer.add_scalar('Test Weight Recall', acc["weightRec"], epoch)

        print("Test Macro F1",acc["macroF1"])
        writer.add_scalar('Test Macro F1', acc["macroF1"], epoch)
        if acc["macroF1"]>bestMacroF1:
          bestMacroF1 = acc["macroF1"]
          modelname = logPath +"/bestMacroF1Model.pth"
          torch.save(model.state_dict(), modelname)
        writer.add_scalar('Test Micro F1', acc["microF1"], epoch)
        if acc["microF1"]>bestMicroF1:
          bestMicroF1 = acc["microF1"]
          modelname = logPath +"/bestMicroF1Model.pth"
          torch.save(model.state_dict(), modelname)
        writer.add_scalar('Test Weight F1', acc["weightF1"], epoch)
        if acc["weightF1"]>bestWeightF1:
          bestWeightF1 = acc["microF1"]
          modelname = logPath +"/bestWeightF1Model.pth"
          torch.save(model.state_dict(), modelname)

        writer.add_scalar('Test Macro IoU', acc["IoU"], epoch)
        print("Test Macro IoU",acc["IoU"])
    if schedulerLR=="rop":
        scheduler.step(sum(testLossF) / testBatches)
    else:
        scheduler.step()


Training for epoch: 1


100%|██████████| 87/87 [00:32<00:00,  2.68it/s]


Completed epoch: 1
Validating model


c:\Users\UZAIR KABEER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\UZAIR KABEER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Macro Precision 0.3909890450192064
Test Macro Recall 0.3909890450192064
Test Macro F1 0.2906184205683941
Test Macro IoU 0.2383595678735855
Training for epoch: 2


...

Completed epoch: 34
Validating model


c:\Users\UZAIR KABEER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\UZAIR KABEER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Macro Precision 0.7636761532686068
Test Macro Recall 0.7636761532686068
Test Macro F1 0.744570676303196
Test Macro IoU 0.6468721352934909
Training for epoch: 35


100%|██████████| 87/87 [00:33<00:00,  2.56it/s]


Completed epoch: 35
Validating model
Test Macro Precision 0.6949188971018664
Test Macro Recall 0.6949188971018664
Test Macro F1 0.6035286959097991
Test Macro IoU 0.5122648042228034


In [23]:
# save the model
modelname = logPath +"/finalModel.pth"
torch.save(model.state_dict(), modelname)
print("Model saved to",modelname)
print("Best Macro F1",bestMacroF1)
print("Best Micro F1",bestMicroF1)
print("Best Weight F1",bestWeightF1)
writer.close()
print("Training completed")

Model saved to ./final_logs/finalModel.pth
Best Macro F1 0.744570676303196
Best Micro F1 0.9632898799635855
Best Weight F1 0.9632898799635855
Training completed


In [25]:
# loading the model
model = ResidualAttentionUNet(11, 11)
model.load_state_dict(torch.load(modelname))
model.eval()
model.to(device)

# testing the model
model.eval()
testBatches = 0
yTrue = []
yPredicted = []
testLossF = []

with torch.no_grad():
    for (image, target) in testLoader:
        image = image.to(device)
        target = target.to(device)
        logits = model(image)
        loss = criterion(logits, target)
        logits = torch.movedim(logits, (0,1,2,3), (0,3,1,2))
        logits = logits.reshape((-1,11))
        target = target.reshape(-1)
        mask = target != -1
        logits = logits[mask]
        target = target[mask]
        probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
        target = target.cpu().numpy()
        testBatches += target.shape[0]
        testLossF.append((loss.data*target.shape[0]).tolist())
        yPredicted += probs.argmax(1).tolist()
        yTrue += target.tolist()
    yPredicted = np.asarray(yPredicted)
    yTrue = np.asarray(yTrue)
    acc = Evaluation(yPredicted, yTrue)
    print("Test Macro Precision",acc["macroPrec"])
    print("Test Micro Precision",acc["microPrec"])
    print("Test Weight Precision",acc["weightPrec"])
    print("Test Macro Recall",acc["macroPrec"])
    print("Test Micro Recall",acc["microRec"])
    print("Test Weight Recall",acc["weightRec"])
    print("Test Macro F1",acc["macroF1"])
    print("Test Micro F1",acc["microF1"])
    print("Test Weight F1",acc["weightF1"])
    print("Test Macro IoU",acc["IoU"])
    print("Test Subset Accuracy",acc["subsetAcc"])

Test Macro Precision 0.6949188971018664
Test Micro Precision 0.9422530056029507
Test Weight Precision 0.9621076301213273
Test Macro Recall 0.6949188971018664
Test Micro Recall 0.9422530056029507
Test Weight Recall 0.9422530056029507
Test Macro F1 0.6035286959097991
Test Micro F1 0.9422530056029507
Test Weight F1 0.9472755078370063
Test Macro IoU 0.5122648042228034
Test Subset Accuracy 0.9422530056029507


In [42]:
unique_classes = np.unique(np.concatenate((yTrue, yPredicted)))
labels = [str(c) for c in unique_classes]

# confusion matrix
conf_matrix = confusion_matrix(yTrue, yPredicted, labels)
print(conf_matrix)


                0     1       2     3       4        5        6         7  \
0           546.0   0.0     7.0   2.0    83.0      0.0    430.0       0.0   
1             0.0  34.0  1133.0   0.0     0.0      0.0      0.0       0.0   
2             9.0   0.0   224.0   1.0     0.0      0.0    151.0       0.0   
3            55.0   0.0     2.0  16.0     0.0      0.0     17.0       0.0   
4            39.0   0.0     0.0  21.0  1114.0     36.0     62.0       0.0   
5             0.0   0.0     0.0   0.0    88.0  15898.0   2889.0       0.0   
6            33.0   0.0     1.0   0.0   131.0    143.0  22486.0       0.0   
7           879.0   0.0     0.0   0.0     0.0      0.0      1.0  124673.0   
8             3.0   0.0     0.0   0.0     3.0      0.0      1.0       7.0   
9             0.0   0.0     0.0   0.0     0.0      4.0    824.0       0.0   
10            0.0   0.0     0.0   0.0     0.0      0.0      4.0       0.0   
Sum        1564.0  34.0  1367.0  40.0  1419.0  16081.0  26865.0  124680.0   